In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [2]:
df = pd.read_csv("../data/processed/datos_limpios_diagnosticos.csv",
                 sep=";", encoding="latin-1")


# 1. Análisis de contenido de detalle de atención


In [3]:
atencion_prueba = df["detalle_atencion"][0]


In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

## Tokenizacion de Oraciones y Palabras

In [5]:
# Tokeniza por oracion y palabras
separado_por_oracion = sent_tokenize(atencion_prueba)
separado_por_palabra = word_tokenize(atencion_prueba)

print(separado_por_oracion)
print(separado_por_palabra)

['Paciente nueva primera consulta de Linares\nderivada del HLCM.', 'paso a 3 medio (Quiere estudiar párvulos).', 'Con diagnostico de  EP critica.', 'Se intenta dilatación percutanea a los 3 ddv sin lograr apertura.', 'Se reaiza cirugia en CEC comprobandose atresia pulmonar sin CIV y VD tripartito e insuficiencia tricuspidea moderada.', 'Se realiza valvulotomia pulmonar mas plastia tricuspidea y plastia auricular el 30-9-1999(Dr. Leon).', 'Evolucionando con Estenosis Residual moderada dilatacion de cavidades derechas y fx sistolica niormal.', 'Trae RNM UC 2013:  VFD VD 108.6 ML/M2.', 'IP FR 33.7% (Examen previo 20111: 15.4%).', 'PRESENTO SINCOPE (Unico episodio) CON ESTUDIO NEGATIVO EN TIL TEST.', 'Eco hoy 12-1-16 Dr. Baraona: Estenosis pulmonar valvular y supravalvular leves con insuficiencia libre.', 'Dilatación e hipertrofia moderada del VD con fx sistolica disminuida.', 'Insuficiencia tricúspide moderada a severa.', 'Hace vida normal en gral, deporte al menos 4 veces por semana, ha 

## Filtro por StopWords

In [6]:
stop_words = stopwords.words("spanish")
atencion_sin_stopwords = [palabra for palabra in separado_por_palabra if palabra not in stop_words]
print(atencion_sin_stopwords)

['Paciente', 'nueva', 'primera', 'consulta', 'Linares', 'derivada', 'HLCM', '.', 'paso', '3', 'medio', '(', 'Quiere', 'estudiar', 'párvulos', ')', '.', 'Con', 'diagnostico', 'EP', 'critica', '.', 'Se', 'intenta', 'dilatación', 'percutanea', '3', 'ddv', 'lograr', 'apertura', '.', 'Se', 'reaiza', 'cirugia', 'CEC', 'comprobandose', 'atresia', 'pulmonar', 'CIV', 'VD', 'tripartito', 'insuficiencia', 'tricuspidea', 'moderada', '.', 'Se', 'realiza', 'valvulotomia', 'pulmonar', 'mas', 'plastia', 'tricuspidea', 'plastia', 'auricular', '30-9-1999', '(', 'Dr.', 'Leon', ')', '.', 'Evolucionando', 'Estenosis', 'Residual', 'moderada', 'dilatacion', 'cavidades', 'derechas', 'fx', 'sistolica', 'niormal', '.', 'Trae', 'RNM', 'UC', '2013', ':', 'VFD', 'VD', '108.6', 'ML/M2', '.', 'IP', 'FR', '33.7', '%', '(', 'Examen', 'previo', '20111', ':', '15.4', '%', ')', '.', 'PRESENTO', 'SINCOPE', '(', 'Unico', 'episodio', ')', 'CON', 'ESTUDIO', 'NEGATIVO', 'EN', 'TIL', 'TEST', '.', 'Eco', 'hoy', '12-1-16', 'Dr.'

## Stemming

In [7]:
# I was taking a ride in the car.
# I was riding  in the car.
# En este caso, ride y riding significan lo mismo. Con stemming se reduce a la raiz de una palabra.

In [8]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()

In [9]:
palabras_prueba = ["python", "pythoner", "pythoning", "pythoned", "pythonly"]
for w in palabras_prueba:
    print(ps.stem(w))

python
python
python
python
pythonli


In [10]:
nuevo_texto = (
    "It is very important to be pythonly while you are pythoning with python. "
    "All pythoners have pythoned poorly at least once."
)

palabras = word_tokenize(nuevo_texto)
for palabra in palabras:
    print(ps.stem(palabra))

it
is
veri
import
to
be
pythonli
while
you
are
python
with
python
.
all
python
have
python
poorli
at
least
onc
.


## Part of Speech

Agrega una etiqueta a cada palabra, indicando si la palabra es un articulo, sustantivo, adjetivo,
etc.

In [11]:
import nltk

from nltk.corpus import state_union
from nltk.tokenize import PunktSentenceTokenizer

- CC: It is the conjunction of coordinating
- CD: It is a digit of cardinal
- DT: It is the determiner
- EX: Existential
- FW: It is a foreign word
- IN: Preposition and conjunction
- JJ: Adjective
- JJR and JJS: Adjective and superlative
- LS: List marker
- MD: Modal
- NN: Singular noun
- NNS, NNP, NNPS: Proper and plural noun
- PDT: Predeterminer
- WRB: Adverb of wh
- WP$: Possessive wh
- WP: Pronoun of wh
- WDT: Determiner of wp
- VBZ: Verb
- VBP, VBN, VBG, VBD, VB: Forms of verbs
- UH: Interjection
- TO: To go
- RP: Particle
- RBS, RB, RBR: Adverb
- PRP, PRP$: Pronoun personal and professional

In [12]:
palabras_con_tag = nltk.pos_tag(atencion_sin_stopwords)
print(palabras_con_tag)

[('Paciente', 'NNP'), ('nueva', 'CC'), ('primera', 'NN'), ('consulta', 'NN'), ('Linares', 'NNP'), ('derivada', 'VBZ'), ('HLCM', 'NNP'), ('.', '.'), ('paso', 'NN'), ('3', 'CD'), ('medio', 'NN'), ('(', '('), ('Quiere', 'NNP'), ('estudiar', 'RB'), ('párvulos', 'NN'), (')', ')'), ('.', '.'), ('Con', 'NNP'), ('diagnostico', 'NN'), ('EP', 'NNP'), ('critica', 'NN'), ('.', '.'), ('Se', 'NNP'), ('intenta', 'JJ'), ('dilatación', 'NN'), ('percutanea', 'NN'), ('3', 'CD'), ('ddv', 'NN'), ('lograr', 'NN'), ('apertura', 'NN'), ('.', '.'), ('Se', 'NNP'), ('reaiza', 'NN'), ('cirugia', 'NN'), ('CEC', 'NNP'), ('comprobandose', 'NN'), ('atresia', 'NN'), ('pulmonar', 'NN'), ('CIV', 'NNP'), ('VD', 'NNP'), ('tripartito', 'NN'), ('insuficiencia', 'NN'), ('tricuspidea', 'NN'), ('moderada', 'NN'), ('.', '.'), ('Se', 'NNP'), ('realiza', 'NN'), ('valvulotomia', 'NN'), ('pulmonar', 'NN'), ('mas', 'NN'), ('plastia', 'NN'), ('tricuspidea', 'NN'), ('plastia', 'NN'), ('auricular', 'JJ'), ('30-9-1999', 'JJ'), ('(', '('

## Chunking

In [15]:
chunkgram = r""""Chunk: {<NNP.?>+<NN.?>}"""
chunkParser = nltk.RegexpParser(chunkgram)
chunked = chunkParser.parse(palabras_con_tag)

In [16]:
chunked

## Chinking

Indica todo lo que se quiere dejar fuera de los chunks

In [20]:
chunkgram = r""""Chunk: {<.*>+}
                        }<VB.?|IN|DT>+{"""
chunkParser = nltk.RegexpParser(chunkgram)
chunked = chunkParser.parse(palabras_con_tag)

In [21]:
chunked

## Named Entity Recognition

Identifica entidades dentro de un texto. Las entidades corresponden a organizaciones (Un Hospital),
personas, fechas, porcentajes, localizaciones, entre otras.

In [25]:
entidades_en_texto = nltk.ne_chunk(palabras_con_tag, binary=True)

In [28]:
entidades_en_texto